In [1]:
import requests
import urllib.request, json 
import pandas as pd
import altair as alt
import numpy as np
from io import StringIO
from Bio import SeqIO
import gzip

In [2]:
# Get constellation annotations from github.com/cov-lineages/constellations/main

def get_json(url_to_json):
    with urllib.request.urlopen(url_to_json) as url:
        data = json.loads(url.read().decode())
    return(data)

user = "cov-lineages"
repo = "constellations"
lineages = {}
genome = {}
url = "https://api.github.com/repos/{}/{}/git/trees/main?recursive=1".format(user, repo)
r = requests.get(url)
res = r.json()
for file in res["tree"]:
    if file['path'].startswith('constellations/definitions/'):
        data = get_json('https://raw.githubusercontent.com/cov-lineages/constellations/main/{}'.format(file['path']))
        lineages[data['label']]=data
    if file['path'].startswith('constellations/data/SARS-CoV-2.json'):
        genome = get_json('https://raw.githubusercontent.com/cov-lineages/constellations/main/{}'.format(file['path']))       

In [3]:
# This generates a list of all gene names in all cov-lineages json blobs

genes = []
for key in lineages.keys():
    for site in lineages[key]['sites']:
        genes.append(site.split(':')[0].lower())

In [4]:
# A sorted set of gene names

sorted(set(genes))

['1ab',
 '8',
 'del',
 'e',
 'm',
 'n',
 'nsp12',
 'nsp13',
 'nsp15',
 'nsp2',
 'nsp3',
 'nsp4',
 'nsp5',
 'nsp6',
 'nsp7',
 'nuc',
 'orf1a',
 'orf1ab',
 'orf1b',
 'orf3a',
 'orf7a',
 'orf8',
 's',
 'spike']

In [5]:
# Here we define standard names
standard_names = 'del,e,m,n,nsp12,nsp13,nsp14,nsp15,nsp2,nsp3,nsp4,nsp5,nsp6,nsp7,nuc,orf1a,orf1ab,orf1b,orf3a,orf7a,orf8,orf3b,s,nuc'.split(',')

In [6]:
# Cheking which names are non-standard in the json files

for name in set(genes):
    if name not in standard_names:
        print(name)

8
1ab
spike


In [7]:
# A translation for non-standard names

non_standard_translations = {'spike':'s','8':'orf8','1ab':'orf1ab'}

In [8]:
# Relabel gene identifiers for variable sites using 
# standard_names and non_standard_translations dicts

for lineage in lineages.keys():
    sites = []
    for site in lineages[lineage]['sites']:
        elements = site.split(':')
        elements[0] = elements[0].lower()
        if elements[0] not in standard_names:
            if elements[0] in non_standard_translations.keys():
                elements[0] = non_standard_translations[elements[0]]
            else:
                print("Non standard gene name: '{}'. Add this to non_standard_translations dict.".format(g))
        sites.append(':'.join(elements))
    lineages[lineage]['sites'] = sites

In [9]:
# URLs for annotations and sequence

fa_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/858/895/GCF_009858895.2_ASM985889v3/GCF_009858895.2_ASM985889v3_genomic.fna.gz"
gff_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/858/895/GCF_009858895.2_ASM985889v3/GCF_009858895.2_ASM985889v3_genomic.gff.gz"

In [10]:
# Function for reading gzipped filed from a URL
# a returning a text blob

def read_gzipped_url(url):
    try:
        with urllib.request.urlopen(url) as response:
            with gzip.open(response,mode='rt') as uncompressed:
                content = uncompressed.read()
    except Exception as e:
        print(e)
    return(content)

In [11]:
import re

# Extract rows from NCBI gff annotation of SARS-CoV-2
# Rows must have either 'mature_protein_region_of_CDS' or
# 'CDS'
# Note that because some rows are repeated we are using python `set`
# Rows containing `mature_protein_region_of_CDS`
pr = re.compile("NC_045512\.2\tRefSeq\tmature_protein_region_of_CDS\t(\d*)\t(\d*).*product=([\w\s\'-]*)\;")

# Rows containing `CDS`
gn = re.compile("NC_045512\.2\tRefSeq\tCDS\t(\d*)\t(\d*).*gene=([\w\s\'-]*)\;")

# Read GFF
gff = read_gzipped_url(gff_url)

# Find matches and save then as a flattened list
matches = set([ x for y in [pr.findall(gff), gn.findall(gff)] for x in y ])

In [12]:
matches

{('10055', '10972', '3C-like proteinase'),
 ('10973', '11842', 'nsp6'),
 ('11843', '12091', 'nsp7'),
 ('12092', '12685', 'nsp8'),
 ('12686', '13024', 'nsp9'),
 ('13025', '13441', 'nsp10'),
 ('13442', '13468', 'RNA-dependent RNA polymerase'),
 ('13442', '13480', 'nsp11'),
 ('13468', '16236', 'RNA-dependent RNA polymerase'),
 ('13468', '21555', 'ORF1ab'),
 ('16237', '18039', 'helicase'),
 ('18040', '19620', "3'-to-5' exonuclease"),
 ('19621', '20658', 'endoRNAse'),
 ('20659', '21552', "2'-O-ribose methyltransferase"),
 ('21563', '25384', 'S'),
 ('25393', '26220', 'ORF3a'),
 ('26245', '26472', 'E'),
 ('26523', '27191', 'M'),
 ('266', '13468', 'ORF1ab'),
 ('266', '13483', 'ORF1ab'),
 ('266', '805', 'leader protein'),
 ('2720', '8554', 'nsp3'),
 ('27202', '27387', 'ORF6'),
 ('27394', '27759', 'ORF7a'),
 ('27756', '27887', 'ORF7b'),
 ('27894', '28259', 'ORF8'),
 ('28274', '29533', 'N'),
 ('29558', '29674', 'ORF10'),
 ('806', '2719', 'nsp2'),
 ('8555', '10054', 'nsp4')}

In [13]:
# Read genome sequence from NCBI
genome = SeqIO.read(StringIO(read_gzipped_url(fa_url)),"fasta")

In [14]:
# Pango lineages uses a specific list of gene names
# This dict is used to translate names obtained from NCBI's gff 
# Info pango-compatible list

ncbi2pango = {
    'endornase':'nsp15', 
     'nsp2':'nsp2', 
     'helicase':'nsp13', 
     'rna-dependent rna polymerase':'nsp12', 
     'nsp3':'nsp3', 
     'm':'m', 
     'orf8':'orf8', 
     "3'-to-5' exonuclease":'nsp14', 
     'orf1ab':'orf1ab', 
     'nsp4':'nsp4', 
     'orf7b':'orf7b', 
     'n':'n', 
     'nsp10':'nsp10', 
     "2'-o-ribose methyltransferase":'nsp16', 
     'e':'e', 
     'nsp7':'nsp7', 
     'nsp9':'nsp9', 
     'nsp8':'nsp8', 
     'orf3a':'orf3a', 
     'nsp11':'nsp11', 
     'nsp6':'nsp6', 
     'leader protein':'nsp1', 
     's':'s', 
     'orf10':'orf10', 
     'orf6':'orf6', 
     '3c-like proteinase':'nsp5', 
     'orf7a':'orf7a'
}

In [15]:
# Translate gene names from ncbi to pango
# using ncbi2pango dict

annotations = {}
for item in matches:
    start, end, gene = item
    gene = gene.lower()
    if not gene in ncbi2pango.keys():
        print('Missing gene {} in ncbi2pango mapping'.format(gene))
    gene = ncbi2pango[gene]
    if gene not in annotations.keys():
        annotations[ gene ] = [ [ int(start), int(end) ] ]
    else:
        annotations[ gene ].append( [ int(start),int(end) ] )

In [16]:
annotations

{'nsp6': [[10973, 11842]],
 'nsp8': [[12092, 12685]],
 'nsp13': [[16237, 18039]],
 'orf6': [[27202, 27387]],
 'nsp4': [[8555, 10054]],
 'orf1ab': [[266, 13483], [13468, 21555], [266, 13468]],
 'nsp9': [[12686, 13024]],
 'orf10': [[29558, 29674]],
 'nsp1': [[266, 805]],
 'orf8': [[27894, 28259]],
 'nsp16': [[20659, 21552]],
 'orf7a': [[27394, 27759]],
 'nsp7': [[11843, 12091]],
 's': [[21563, 25384]],
 'nsp12': [[13468, 16236], [13442, 13468]],
 'nsp5': [[10055, 10972]],
 'nsp3': [[2720, 8554]],
 'nsp15': [[19621, 20658]],
 'nsp14': [[18040, 19620]],
 'm': [[26523, 27191]],
 'n': [[28274, 29533]],
 'orf3a': [[25393, 26220]],
 'e': [[26245, 26472]],
 'nsp10': [[13025, 13441]],
 'nsp11': [[13442, 13480]],
 'orf7b': [[27756, 27887]],
 'nsp2': [[806, 2719]]}

In [17]:
# Hacky tweaks to add orf1a and orf1b
# and to adjust RdRp to full span (before and after franeshift, decreasing start by 1:
# it goes form [[13442, 13468], [13468, 16236]] to [[13441, 16236]]

annotations['orf1a']=[[266,13468]]
annotations['orf1b']=[[13468,21555]]
annotations['nsp12']=[[13441, 16236]]

In [18]:
annotations

{'nsp6': [[10973, 11842]],
 'nsp8': [[12092, 12685]],
 'nsp13': [[16237, 18039]],
 'orf6': [[27202, 27387]],
 'nsp4': [[8555, 10054]],
 'orf1ab': [[266, 13483], [13468, 21555], [266, 13468]],
 'nsp9': [[12686, 13024]],
 'orf10': [[29558, 29674]],
 'nsp1': [[266, 805]],
 'orf8': [[27894, 28259]],
 'nsp16': [[20659, 21552]],
 'orf7a': [[27394, 27759]],
 'nsp7': [[11843, 12091]],
 's': [[21563, 25384]],
 'nsp12': [[13441, 16236]],
 'nsp5': [[10055, 10972]],
 'nsp3': [[2720, 8554]],
 'nsp15': [[19621, 20658]],
 'nsp14': [[18040, 19620]],
 'm': [[26523, 27191]],
 'n': [[28274, 29533]],
 'orf3a': [[25393, 26220]],
 'e': [[26245, 26472]],
 'nsp10': [[13025, 13441]],
 'nsp11': [[13442, 13480]],
 'orf7b': [[27756, 27887]],
 'nsp2': [[806, 2719]],
 'orf1a': [[266, 13468]],
 'orf1b': [[13468, 21555]]}

In [19]:
# Recompute coordionates from protein-based
# to genome based

def prot_to_genome(sites,annotations):
    
    '''
    
    Takes sites as list in the following format:
        ['nuc:T733C','nuc:C2749T','orf1ab:S1188L']
    and annotations in the following format:
            {'nsp15': [[19621, 20658]],'nsp2': [[806, 2719]] ... }
            
    Returns a dict with original site names and their genomic coordinates:
    
        {'nuc:T733C': 732,'nuc:C2749T': 2748,'orf1ab:S1188L': 3826 ...}

    '''
    
    p = re.compile("[A-Z]*(\d*)")
    voc = {}
    for site in sites:
        cols = site.split(':')
        
        # The line below is based on assumpltion that coordinates annotated as belonging to orf1ab
        # actually belong to orf1a
        
        if cols[0] == 'orf1ab':
            cols[0] = 'orf1a'
        if not site.startswith('del') and not site.startswith('nuc'):
            start = annotations[cols[0]][0][0]
            voc[site] = (int(p.findall(cols[1])[0]))*3+start-4
        elif site.startswith('del'):
            voc[site] = int(cols[1])-1
        elif site.startswith('nuc'):
            voc[site] = int(p.findall(cols[1])[0])-1
    return(voc)

In [20]:
def validate_against_genome(genome,sites):

    '''
    
    Validates aa replacement sites sites across genome by excising a codon,
    translating it, and comparing translation with annotation
    
    Takes genome as BioPython SeqRecord and sites as a dict:
    
    {'nuc:T733C': 732,'nuc:C2749T': 2748,'orf1ab:S1188L': 3826 ...}
    

    '''
    problems = {}
    for key,site in sites.items():
        if not ( key.startswith('del') or key.startswith('nuc') ):
            aa_voc = key.split(':')[1][0]
            aa = genome[site:site+3].translate()[0]
            if not aa==aa_voc:
                problems[key]=[site,aa,aa_voc,aa==aa_voc]
    return(problems)
                

In [21]:
# Here coordinates are converted from protein-based to genome based
# Amino acid replacements are validated by translating a corresponding genomic region
# and comparing with annotation

genomic_sites = {}
for lineage in lineages.keys():
    genomic_sites[lineage] = prot_to_genome(lineages[lineage]['sites'],annotations)
    problems = validate_against_genome(genome, genomic_sites[lineage])
    if len(problems)>0:
        print(lineage,problems)

In [22]:
def expand_coordinates(sites):

    '''
        This function expands coordinates. A codon beginning position is exapanded into three positions
        corresponding to all there codon nucleotides. For example, if codon begins at 4939 it will be expanded
        into three lines:
        4939   nsp3:L741F  A.23.1-like+E484K
        4940   nsp3:L741F  A.23.1-like+E484K
        4941   nsp3:L741F  A.23.1-like+E484K
        
        Input:
            A dict with sites like this:
            
            {'nsp3:L741F': 4939,
             'nsp6:M86I': 11227}
             
        Output:
            A dict with the following structure:
            
            {4939: 'nsp3:L741F',
             4940: 'nsp3:L741F',
             4941: 'nsp3:L741F',
             11227: 'nsp6:M86I',
             11228: 'nsp6:M86I',
             11229: 'nsp6:M86I'}
    '''

    pattern = "([A-Z]+)\d+[A-Z\*-]+"
    a = re.compile(pattern)
    expanded_list = {}
    for key,site in sites.items():
        if key.startswith('del'):
            l = int(key.split(':')[2])
            for pos in np.arange(site, site+l):
                expanded_list[pos]=key
        elif key.startswith('nuc'):
            try:
                l = len(a.findall(key.split(':')[1])[0])
            except:
                print('Problem matching {0} to {1}. Assigning as {2}:{0}'.format(key,pattern,site))
                expanded_list[site]=key
            for pos in np.arange(site, site+l):
                expanded_list[pos]=key
        else:
            try:
                l = len(a.findall(key)[0])
            except:
                print('Problem matching {0} to {1}. Assigning as {2}:{0}'.format(key,pattern,site))
                expanded_list[site]=key
            for pos in np.arange(site,site+l*3):
                expanded_list[pos]=key 
    return(expanded_list)

In [23]:
# Run expansion on all lineages

exp = {'POS':[],'variant':[],'lineage':[]}
for key in genomic_sites.keys():
    for pos,var in expand_coordinates(genomic_sites[key]).items():
        #print(pos,var)
        exp['POS'].append(pos)
        exp['variant'].append(var)
        exp['lineage'].append(key)

Problem matching nuc:28262+AACA to ([A-Z]+)\d+[A-Z\*-]+. Assigning as 28261:nuc:28262+AACA


In [24]:
# Create a dataframe
df = pd.DataFrame.from_dict(exp)

In [25]:
alt.renderers.set_embed_options(actions=True)

alt.Chart(df.groupby(['lineage','variant']).agg({'POS':'min'}).reset_index()).mark_tick(color='green').encode(
    x=alt.X("POS:Q",title='Position',axis=alt.Axis(grid=False)),
    y=alt.Y("lineage:O",title='Lineage',
            axis=alt.Axis(grid=False)),
    tooltip=[
            alt.Tooltip("variant",title="variant")]
).properties(
    width=800,
    height=400
)

alt.Chart(...)

In [26]:
df.to_csv('constellations.csv.gz',index=False,compression='gzip')